<a href="https://colab.research.google.com/github/mudu89/ML_AssignmentPS01/blob/main/2024DA04200_Problem_Statement_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
